In [38]:
import os
import glob
import shutil
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LightSource
import rasterio.plot
import os
import seaborn as sns


In [ ]:
print(os.getcwd())

In [16]:
def rel(path_str):
    return str((Path().resolve().parent / path_str).resolve())

In [17]:
# input sample paths
# the order is importent
BASE_PATHS = [
    '../demo/2021-02',
    '../demo/2024-05'
]

# input paths inside of an each sample path
INPUT_RESULT_FOLDER = 'output/analyser'
INPUT_DB = 'db/database.gpkg'

# output folder - will include comparisons
OUTPUT_CSV_FOLDER = '../demo/differences'

os.makedirs(OUTPUT_CSV_FOLDER, exist_ok=True)

In [18]:
def clean_folder(path):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.mkdir(path)     

paths = BASE_PATHS
names = list(map(lambda p: os.path.basename(p), paths))

clean_folder(OUTPUT_CSV_FOLDER)

In [19]:
# analyser results
samples = []

for path in paths:
    results_file = glob.glob(os.path.join(path, INPUT_RESULT_FOLDER, '*.csv'))
    if len(results_file) == 0:
        break
    res = pd.read_csv(results_file[0], encoding='utf-8', sep=',', skipinitialspace=True)
    samples.append(res)    

In [20]:
# distance between points 
db_file = os.path.join(paths[0], INPUT_DB)
profiles = gpd.read_file(db_file, layer='profiles')
points_distance = round(profiles.iloc[0].geometry.distance(profiles.iloc[1].geometry), 3)

In [ ]:
print("Samples:", len(samples))
for i, s in enumerate(samples):
    print(f"Sample {i}:", s.shape if hasattr(s, 'shape') else type(s))


In [22]:
def get_results_diff(first, second):
    merged = first.merge(second, left_on='profile_id', right_on='profile_id', how='left')
    diff = pd.DataFrame({
        'profile_id': merged.profile_id,
        'zero_position': (merged.last_zero_id_y - merged.last_zero_id_x) * points_distance,
        'zero_elevation': merged.last_zero_elevation_y - merged.last_zero_elevation_x,

        'bottom_position': (merged.bottom_id_y - merged.bottom_id_x) * points_distance,
        'bottom_elevation': merged.bottom_elevation_y - merged.bottom_elevation_x,

        'top_position': (merged.top_id_y - merged.top_id_x) * points_distance,
        'top_elevation': merged.top_elevation_y - merged.top_elevation_x,

        'beach_width': merged.beach_width_y - merged.beach_width_x,
        'beach_slope': merged.beach_slope_y - merged.beach_slope_x,
        'beach_volume': merged.beach_volume_y - merged.beach_volume_x,

        'dune_width': merged.dune_width_y - merged.dune_width_x,
        'dune_slope': merged.dune_slope_y - merged.dune_slope_x,
        'dune_volume': merged.dune_volume_y - merged.dune_volume_x  
    })
    return diff

In [23]:
diffs = []

for idx, sample in enumerate(samples[:-1]):
    diffs.append(get_results_diff(samples[idx], samples[idx+1]))

In [24]:
fields = [
    'zero_position',
    'zero_elevation',
    'bottom_position',
    'bottom_elevation',
    'top_position',
    'top_elevation',
    'beach_width',
    'beach_slope',
    'beach_volume',
    'dune_width',
    'dune_slope',
    'dune_volume'
]

for field in fields:
    field_diffs = pd.DataFrame({
        'profile_id': diffs[0].profile_id
    })
    for diff_idx, diff in enumerate(diffs):
        field_diffs = field_diffs.merge(diff[['profile_id', field]], left_on='profile_id', right_on='profile_id', how='left')
        field_diffs.rename(columns = {field: f'{names[diff_idx]}_{names[diff_idx+1]}'}, inplace = True)
    
    field_diffs.to_csv(os.path.join(OUTPUT_CSV_FOLDER, f'{field}_diff.csv'), encoding='utf-8', sep=',')

In [25]:
def plot_field_differences(field_diffs, field_name, output_folder, save=True, show=True):
    """
    Plots a line chart for differences.
    """

    friendly_names = {
        'beach_width': 'Beach width',
        'beach_slope': 'Beach slope',
        'beach_volume': 'Beach volume',
        'dune_width': 'Dune width',
        'dune_slope': 'Dune slope',
        'dune_volume': 'Dune volume'
    }
    
    friendly_label = friendly_names.get(field_name, field_name.replace('_', ' ').capitalize())

    plt.figure(figsize=(10, 6))
    data_column = [col for col in field_diffs.columns if col != 'profile_id'][0]
    plt.plot(field_diffs['profile_id'], field_diffs[data_column], label=data_column)

    plt.title(f'Differences in {friendly_label}')
    plt.xlabel('Profile ID')
    plt.ylabel('Difference')
    plt.xticks(field_diffs['profile_id'])
    plt.legend(loc='upper left')
    plt.grid(linestyle=':')

    if save:
        output_file = os.path.join(output_folder, f'{field_name}_differences_plot.jpg')
        plt.savefig(output_file, bbox_inches='tight', dpi=300)
        print(f'Plot saved: {output_file}')
    if show:
        plt.show()
    plt.close()


# Basic plots

In [ ]:
for field in fields:
    field_diff_path = os.path.join(OUTPUT_CSV_FOLDER, f'{field}_diff.csv')
    field_diffs = pd.read_csv(field_diff_path, encoding='utf-8', sep=',', index_col=0)
    field_diffs['profile_id'] = field_diffs['profile_id'].astype(int)
    field_diffs = field_diffs.sort_values('profile_id')
    plot_field_differences(field_diffs, field, OUTPUT_CSV_FOLDER)


# Plots with coordinates

In [30]:
def plot_differences_on_map(field_diffs, field_name, profiles_gdf, output_folder, transect_width=50, save=True, show=True):
    """
    Plots a map of differences based on profile geometry, marking the centres of profiles with labels, taking into account the width of transects.
    """
    # Name dictionary
    friendly_names = {
        'beach_width': 'Beach width',
        'beach_slope': 'Beach slope',
        'beach_volume': 'Beach volume',
        'dune_width': 'Dune width',
        'dune_slope': 'Dune slope',
        'dune_volume': 'Dune volume'
    }

    # Column alignment
    profiles_gdf.rename(columns={'no_transect': 'profile_id'}, inplace=True)

    # Matching profile_id data type
    field_diffs['profile_id'] = field_diffs['profile_id'].astype(int)
    profiles_gdf['profile_id'] = profiles_gdf['profile_id'].astype(int)

    # Removing unnecessary columns
    field_diffs = field_diffs.drop(columns=['Unnamed: 0'], errors='ignore')

    # Resetting indexes
    profiles_gdf = profiles_gdf.reset_index(drop=True)
    field_diffs = field_diffs.reset_index(drop=True)

    # Data connection
    merged_gdf = profiles_gdf.merge(field_diffs, on='profile_id', how='left')

    # Calculation of profile dimensions
    profile_centers = merged_gdf.groupby('profile_id').geometry.apply(lambda geom: geom.unary_union.centroid)

    # Visualisation of differences for each difference column
    for col in field_diffs.columns[1:]:  
        fig, ax = plt.subplots(figsize=(20, 10))  

        # transect width
        merged_gdf['geometry'] = merged_gdf.geometry.buffer(transect_width / 2)  

        # Map
        merged_gdf.plot(
            column=col,
            cmap='coolwarm',
            legend=True,
            legend_kwds={'label': 'Changes', 'orientation': 'vertical'},
            ax=ax
        )

        ax.set_title(f"{friendly_names.get(col, col)} ({friendly_names.get(field_name, field_name)})", fontsize=20)
        ax.set_xlabel("Geographic X Coordinate (EPSG:2180)", fontsize=14)
        ax.set_ylabel("Geographic Y Coordinate (EPSG:2180)", fontsize=14)

        for profile_id, center in profile_centers.items():
            ax.annotate(
                text=str(profile_id),
                xy=(center.x, center.y),
                xytext=(0, 0), 
                textcoords="offset points",
                fontsize=10,
                color="black",
                ha='center',  
                va='center'   
            )

        if save:
            output_file = os.path.join(output_folder, f'{friendly_names.get(col, col).replace(" ", "_")}_{field_name}_map.jpg')
            plt.savefig(output_file, bbox_inches='tight', dpi=300)
            print(f'Map saved: {output_file}')
        if show:
            plt.show()

        plt.clf()
        plt.close('all')


In [31]:
profiles_gdf = gpd.read_file(db_file, layer='profiles')

In [ ]:
for field in ['beach_width', 'beach_slope', 'beach_volume', 'dune_width', 'dune_slope', 'dune_volume']:
    field_diff_path = os.path.join(OUTPUT_CSV_FOLDER, f'{field}_diff.csv')
    field_diffs = pd.read_csv(field_diff_path, encoding='utf-8', sep=',')
    plot_differences_on_map(field_diffs, field, profiles_gdf, OUTPUT_CSV_FOLDER)


# Plots with coordinates and DEM on background

In [34]:
def plot_differences_on_map(field_diffs, field_name, profiles_gdf, output_folder,
                             transect_width=50, save=True, show=True,
                             dem_path=None, dem_scale=4):
    """
    Draws a difference map with a profile buffer and DEM as background
    """

    # Hillshade parameters
    HILLSHADE_AZIMUTH = 300
    HILLSHADE_ALTITUDE = 50
    VMIN_CORRECTION = 0
    VMAX_CORRECTION = 0

    friendly_names = {
        'beach_width': 'Beach width',
        'beach_slope': 'Beach slope',
        'beach_volume': 'Beach volume',
        'dune_width': 'Dune width',
        'dune_slope': 'Dune slope',
        'dune_volume': 'Dune volume'
    }

    profiles_gdf = profiles_gdf.rename(columns={'no_transect': 'profile_id'})
    field_diffs['profile_id'] = field_diffs['profile_id'].astype(int)
    profiles_gdf['profile_id'] = profiles_gdf['profile_id'].astype(int)

    field_diffs = field_diffs.drop(columns=['Unnamed: 0'], errors='ignore')
    profiles_gdf = profiles_gdf.reset_index(drop=True)
    field_diffs = field_diffs.reset_index(drop=True)

    merged_gdf = profiles_gdf.merge(field_diffs, on='profile_id', how='left')
    merged_gdf['geometry_buffered'] = merged_gdf.geometry.buffer(transect_width / 4)
    centroids = merged_gdf.groupby('profile_id').geometry.first().centroid

    # DEM 
    hillshade = None
    dem_extent = None
    if dem_path:
        with rasterio.open(dem_path) as src:
            out_shape = (1, src.height // dem_scale, src.width // dem_scale)
            dem_data = src.read(
                1,
                out_shape=out_shape,
                resampling=rasterio.enums.Resampling.average
            )
            transform = src.transform
            transform = rasterio.Affine(transform.a * dem_scale, transform.b, transform.c,
                                        transform.d, transform.e * dem_scale, transform.f)
            dem_bounds = src.bounds
            dem_extent = [dem_bounds.left, dem_bounds.right, dem_bounds.bottom, dem_bounds.top]

            dem_data[dem_data == src.nodata] = np.nan
            min_elevation = np.nanmin(dem_data)
            dem_data[dem_data < min_elevation] = np.nan

            hillshade = LightSource(azdeg=HILLSHADE_AZIMUTH, altdeg=HILLSHADE_ALTITUDE).hillshade(dem_data, vert_exag=3)

    for col in field_diffs.columns[1:]:
        fig, ax = plt.subplots(figsize=(20, 10))

        if hillshade is not None:
            vmin, vmax = np.nanpercentile(hillshade, (1, 99))
            ax.imshow(
                hillshade,
                extent=dem_extent,
                cmap='gray',
                origin='upper',
                vmin=vmin + VMIN_CORRECTION,
                vmax=vmax + VMAX_CORRECTION,
                zorder=0  
            )

        values = merged_gdf[col]
        cmap = plt.get_cmap('coolwarm')
        norm = plt.Normalize(vmin=values.min(), vmax=values.max())
        colors = cmap(norm(values))
        colors[:, -1] = 0.5

        merged_gdf.set_geometry('geometry_buffered').plot(
            facecolor=colors,
            edgecolor='none',
            ax=ax,
            zorder=1
        )

        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax)
        cbar.set_label('Changes')


        for profile_id, center in centroids.items():
            ax.annotate(str(profile_id), xy=(center.x, center.y), xytext=(0, 0),
                        textcoords="offset points", fontsize=10, color="black",
                        ha='center', va='center')

        ax.set_title(f"{friendly_names.get(col, col)} ({friendly_names.get(field_name, field_name)})", fontsize=20)

        ## Change EPSG code or description
        ax.set_xlabel("X (EPSG:2180)", fontsize=14)
        ax.set_ylabel("Y (EPSG:2180)", fontsize=14)

        if save:
            out_name = f"{friendly_names.get(col, col).replace(' ', '_')}_{field_name}_map.jpg"
            output_file = os.path.join(output_folder, out_name)
            plt.savefig(output_file, bbox_inches='tight', dpi=72)
            print(f'Map saved: {output_file}')
        if show:
            plt.show()

        plt.clf()
        plt.close('all')

In [ ]:
#This script could be time consuming. 
#To check data you can run only one field instead of all
 
#DEM path placed in background
dem_path = '../demo/2024-05/input/dem/low-2024-05-29.tif'

for field in ['beach_width', 'beach_slope', 'beach_volume', 
              'dune_width', 'dune_slope', 'dune_volume']:
    field_diff_path = os.path.join(OUTPUT_CSV_FOLDER, f'{field}_diff.csv')
    field_diffs = pd.read_csv(field_diff_path, encoding='utf-8', sep=',')
    plot_differences_on_map(
        field_diffs=field_diffs,
        field_name=field,
        profiles_gdf=profiles_gdf,
        output_folder=OUTPUT_CSV_FOLDER,
        transect_width=50,
        dem_path=dem_path,
        dem_scale=16, #DEM scale (with lower number files will be bigger and calculation will be more time consuming. A value of 16 means that the resolution will be scaled down 16 times )
        save=True,
        show=True
    )





# Heat maps

In [39]:
def plot_heatmap_of_differences(field_diffs, field_name, output_folder, save=True, show=True):
    """
    Plot heatmaps
    """
    field_diffs = field_diffs.loc[:, ~field_diffs.columns.str.contains('^Unnamed')]

    # Name dictionary
    friendly_names = {
        'beach_width': 'Beach width',
        'beach_slope': 'Beach slope',
        'beach_volume': 'Beach volume',
        'dune_width': 'Dune width',
        'dune_slope': 'Dune slope',
        'dune_volume': 'Dune volume'
    }

    heatmap_data = field_diffs.set_index('profile_id').T
    plt.figure(figsize=(20, 3))
    sns.heatmap(heatmap_data, cmap='coolwarm', annot=False)
    plt.title(f'{friendly_names.get(field_name, field_name)} differences')
    plt.xlabel('Profile ID')
    plt.ylabel('Time Comparison')

    if save:
        output_file = os.path.join(output_folder, f'{field_name}_heatmap.jpg')
        plt.savefig(output_file, bbox_inches='tight', dpi=600)
        print(f'Heatmap saved: {output_file}')
    if show:
        plt.show()
    plt.close()


In [ ]:

fields = [
    'beach_width', 'beach_slope', 'beach_volume', 
    'dune_width', 'dune_slope', 'dune_volume'
]

for field in fields:
    field_diff_path = os.path.join(OUTPUT_CSV_FOLDER, f'{field}_diff.csv')
    field_diffs = pd.read_csv(field_diff_path, encoding='utf-8', sep=',')
    plot_heatmap_of_differences(field_diffs, field, OUTPUT_CSV_FOLDER)
